In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pydicom
import uuid #for generating guid code 
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import regularizers
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split

In [23]:
batch_size = 8
epochs = 200

threshold = 7000 #intensity of pixel to ignore
#number of depth images to keep, only a certain number contains the base gangli
zmin = 20
zmax = 30
#portion of image which contains the interested part of brain scan
xmin = 33
xmax = 162
ymin = 73
ymax = 202

deltaX = xmax-xmin
deltaY = ymax-ymin
deltaZ = zmax-zmin

img_size = deltaX+1

#during data augmentation the images will be shifted in the 4 directions of 5% and 10% of the width
shifting_percentages = [img_size*0.05, img_size*0.1, -img_size*0.05, -img_size*0.1]

background_percentage = 0.4

num_classes = 2

#model hyperparameters
conv_depth_1 = 15 
conv_depth_2 = 25 
conv_depth_3 = 50 
conv_depth_4 = 50 
fc_nodes = 512

drop_prob_1 = 0.25 
drop_prob_2 = 0.5
drop_prob_3 = 0.25 
drop_prob_4 = 0.5
l2_penalty = 1e-3

learning_rate = 1e-3

In [52]:
project_dir = os.getcwd()

try_num = 1
try_dir = os.path.join(project_dir, 'try_{try_num}'.format(try_num=try_num))
if os.path.exists(try_dir):
    raise Exception("Directory name already used, update try_num")
else:
    os.mkdir(try_dir)

data_folder = os.path.join(project_dir, 'CDOPA_dataset')
csv_path = os.path.join(project_dir, 'carbidopa.csv')

hyper_params_filepath = os.path.join(try_dir, 'hyperparameters.txt')
model_filepath = os.path.join(try_dir, 'model.h5')
summary_filepath = os.path.join(try_dir, 'model_summary.txt')
hist_csv_filepath = os.path.join(try_dir, 'history.csv')

In [3]:
"""
DATA LOADING FUNCTIONS
"""

def load_scan(path):
    slices = [pydicom.read_file(os.path.join(path, s), force=True) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices


def get_pixels(slices, rescale=False):
    if rescale:
        image = np.stack([s.pixel_array*(s.RescaleSlope*10) for s in slices])
        return np.array(image,dtype=np.float32)
    else:
        image = np.stack([s.pixel_array for s in slices])
        return np.array(image,dtype=np.int16)
    

def suv(slice, xmin, xmax, ymin, ymax):
    x = np.linspace(xmin, xmax, xmax-xmin+1)
    y = np.linspace(ymin, ymax, ymax-ymin+1)
    X,Y = np.meshgrid(x,y)
    Z = slice[X.astype(np.int),Y.astype(np.int)]
    return X,Y,Z


def slicecutoff(slice, threshold):
    slice[slice<threshold]=0.
    return slice

    
def create_dataset(data_folder, rescale=False, threshold=threshold):

    inumpatients = len(os.listdir(data_folder))#number of datapoints
    X_DATA = np.zeros([inumpatients, deltaX+1, deltaY+1, deltaZ])#initially 'empty dataset'
    patients = os.listdir(data_folder)#list all the folders containing the .dcm for each patient
    patients.sort()
    
    for num_patient, patient in enumerate(patients):
        patient_n = get_pixels(load_scan(os.path.join(data_folder, patients[num_patient])))

        #reference central slice and mask
        slice_central = patient_n[zmin+4]
        slice_centralcut = slice_central[xmin:xmax+1, ymin:ymax+1]

        if rescale:
            threshold_roi = slice_centralcut.max()*background_percentage
            threshold = threshold_roi

        mask_central = slicecutoff(slice_centralcut, threshold)

        for index, slice in enumerate(patient_n):
            if index in range(zmin,zmax):
                
                slicecut = slice[xmin:xmax+1, ymin:ymax+1]
                mask = slicecutoff(slicecut, threshold)

                #here it makes sure the larger portion of the mask is considered background
                #zeroing out the parts outsite the central region of interest
                if index < zmin + 4:
                    if (mask_central-mask).sum() > mask_central.sum():
                        slicecut[True] = 0.
                        mask[True] = 0.
                if index > zmin +4:
                    if abs((mask-mask_central)).sum() > mask_central.sum():
                        slicecut[True] = 0.
                        mask[True] = 0.
                
                X_DATA[num_patient,:,:,index-zmin] = slicecut
    
    return X_DATA
            

In [47]:
"""
DATASET MANIPULATION FUNCTIONS
"""

#shift param is how many pixel the image has to be shifted, 
# it has to be a signed flot for tanslation to the left or right


def shift_image(image, shift_param, axis='x'):
    
    if axis=='x':
        return tfa.image.translate(image, [shift_param,0])

    if axis=='y':
        return tfa.image.translate(image, [0, shift_param])

    else:
        raise Exception("Only x or y are valid axes")


def get_shifted_dataset(X_DATA, Y_D, Y_DATA, shifting_percentages):

    axes = ['x', 'y']

    augmented_dataset = X_DATA

    #augmented_dataset = np.zeros([2*len(shifting_percentages), img_size, img_size, deltaZ])

    for numpatient in range(len(X_DATA)):

        for shift_perc in shifting_percentages:

            for axis in axes:
            
                shifted_image = shift_image(X_DATA[numpatient, :, :, :], shift_param=shift_perc, axis=axis)
                #shifted_image_minus = shift_image(X_DATA[numpatient, :, :, :], shift_param=-shift_perc, axis=axis)
                shifted_image = np.expand_dims(shifted_image, axis=0)
                #shifted_image_minus = np.expand_dims(shifted_image_minus, axis=0)
                augmented_dataset = np.append(augmented_dataset, shifted_image, axis=0)
                #augmented_dataset = np.append(augmented_dataset, shifted_image_minus, axis=0)

                guid = uuid.uuid4()
                current_label = Y_DATA[numpatient]
                last_index = len(X_DATA)
                
                Y_D = np.append(Y_D, np.empty([1,2]), axis=0)
                Y_DATA = np.append(Y_DATA, np.empty([1,1]), axis=0)
                Y_D[last_index + numpatient, :] = (str(guid), current_label)
                Y_DATA[last_index + numpatient] = current_label

    return augmented_dataset, Y_D, Y_DATA


def prep_dataset(X_DATA, Y_DATA, num_classes):

    X_DATA = X_DATA.astype('float32')
    X_DATA = np.expand_dims(X_DATA, axis=4)#axis to store channel info

    Y_DATA[Y_DATA=='N'] = 0.
    Y_DATA[Y_DATA=='T'] = 0.
    Y_DATA[Y_DATA=='P'] = 1.
    Y_DATA = keras.utils.to_categorical(Y_DATA, num_classes)

    return X_DATA, Y_DATA
    

In [5]:
"""
MODEL BUILDING FUNCTIONS
"""

def conv_pool_drop_block(
    input,
    num_filters,
    l2_penalty,
    drop_rate,
    kernel_size=3,
    padding='same',
    kernel_initializer='he_uniform',
    kernel_regularizer=regularizers.L2(l2_penalty),
    activation='relu',
    pool_size=2,
):

    x = layers.Conv3D(
        num_filters, 
        kernel_size=kernel_size,
        padding=padding,
        kernel_initializer=kernel_initializer,
        kernel_regularizer=kernel_regularizer,
        activation=activation
        )(input)

    x = layers.MaxPooling3D(pool_size=pool_size)(x)
    x = layers.Dropout(drop_rate)(x)

    return x


def fc_block(
    input,
    num_layers,
    fc_nodes,
    drop_rate,
    l2_penalty,
    kernel_regularizer=regularizers.L2(l2_penalty),
    activation='relu',

):
    x = layers.Flatten()(input)

    for fc_layers in range(num_layers):
        x = layers.Dense(
            fc_nodes,
            kernel_regularizer=kernel_regularizer,
            )(x)
        x = layers.Dropout(drop_rate)(x)
    
    return x
    

def build_model(
    img_size,
    num_classes,
):
    input = keras.Input(shape=(img_size, img_size, deltaZ, 1))
    x = layers.Rescaling(scale=1/125.5, offset=-1)(input)

    x = conv_pool_drop_block(x, num_filters=conv_depth_1, l2_penalty=l2_penalty, drop_rate=drop_prob_1)
    x = conv_pool_drop_block(x, num_filters=conv_depth_2, l2_penalty=l2_penalty, drop_rate=drop_prob_2)
    x = conv_pool_drop_block(x, num_filters=conv_depth_3, l2_penalty=l2_penalty, drop_rate=drop_prob_3)
    x = layers.Conv3D(conv_depth_4, (3,3,3), padding='same', kernel_initializer='he_uniform', kernel_regularizer=regularizers.L2(l2_penalty), activation='relu')(x)
    x = layers.Dropout(drop_prob_4)(x)

    x = fc_block(x, num_layers=2, fc_nodes=fc_nodes, drop_rate=drop_prob_4, l2_penalty=l2_penalty)
    
    output = layers.Dense(num_classes, kernel_regularizer=regularizers.L2(l2_penalty), activation='softmax')(x)

    return keras.Model(inputs=input, outputs=output)


In [33]:
df_csv = pd.read_csv(csv_path)
df_csv['GUID']= df_csv['GUID'].astype(str)
df_sorted = df_csv.sort_values('GUID')
df_sorted = df_sorted.iloc[0:87,:]#in the csv there are more datapoints than in the CDOPA_dataset folder

Y_D = np.array(df_sorted[['GUID','LABEL']])
Y_DATA = np.array(df_sorted[['LABEL']])
X_DATA = create_dataset(data_folder)

In [35]:
print(X_DATA.shape)
print(Y_D.shape)
print(Y_DATA.shape)

(87, 130, 130, 10)
(87, 2)
(87, 1)


In [36]:
X_DATA, X_DATA_TEST, Y_DATA, Y_DATA_TEST, Y_D, Y_D_TEST = train_test_split(X_DATA, Y_DATA, Y_D, test_size=0.30, random_state=21)

In [37]:
print(X_DATA.shape)
print(Y_D.shape)
print(Y_DATA.shape)

(60, 130, 130, 10)
(60, 2)
(60, 1)


In [18]:
print(X_DATA[0, :, :, :].shape)

(130, 130, 10)


In [38]:
X_DATA_NEW, Y_D_NEW, Y_DATA_NEW = get_shifted_dataset(X_DATA, Y_D, Y_DATA, shifting_percentages=shifting_percentages)

In [39]:
print(X_DATA_NEW.shape)
print(Y_D_NEW.shape)
print(Y_DATA_NEW.shape)

(540, 130, 130, 10)
(540, 2)
(540, 1)


In [49]:
X_DATA_NEW, Y_DATA_NEW = prep_dataset(X_DATA_NEW, Y_DATA_NEW, num_classes=num_classes)

In [51]:
print(X_DATA_NEW.shape)
print(Y_DATA_NEW.shape)

(540, 130, 130, 10, 1)
(540, 2)


In [ ]:
X_DATA_TRAIN, X_DATA_VAL, Y_DATA_TRAIN, Y_DATA_VAL, Y_D_TRAIN, Y_D_VAL = train_test_split(X_DATA_NEW, Y_DATA_NEW, Y_D_NEW, test_size=0.20, random_state=21)

In [ ]:
print(X_DATA_TRAIN.shape)
print(X_DATA_VAL.shape)
print(Y_D_TRAIN.shape)
print(Y_D_VAL.shape)

In [ ]:
loss = keras.losses.CategoricalCrossentropy()
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

model = build_model(img_size=img_size, num_classes=num_classes)

with open(summary_filepath, 'w') as f:
    model.summary(print_fn=lambda x: f.write(x + '\n'))

model.compile(
    loss=loss,
    optimizer=optimizer,
    metrics=['accuracy']
)

callbacks = [
    keras.callbacks.EarlyStopping(
        patience=25,
        monitor='val_loss'
    ),
    keras.callbacks.ModelCheckpoint(
        filepath=model_filepath,
        save_best_only=True
    )
]

history = model.fit(
    x = X_DATA_TRAIN,
    y = Y_DATA_TRAIN,
    batch_size=batch_size,
    validation_data=(X_DATA_VAL, Y_DATA_VAL),
    epochs=epochs,
    callbacks=callbacks,
)

hist_df = pd.DataFrame(history.history)
with open(hist_csv_filepath, mode='w') as f:
    hist_df.to_csv(f)
